In [1]:
import wandb
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
import os


In [2]:
wandb.login(key='905a03c6f175d2e773d5a505f4806f09cd514e03')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ymkim78. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/student/.netrc


True

In [3]:
wandb.init(project='db_hw_3_youngmok_kim')

In [4]:
sweep_config = {
    'method': 'random'
    }

In [5]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'fc_layer_size': {
        'values': [128, 256, 512]
        },
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

In [6]:
parameters_dict.update({
    'epochs': {
        'values': [5, 10]}
    })

In [7]:
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms
        'distribution': 'q_log_uniform_values',
        'q': 8, # value will increment/decrement by intervals of 8 ex. 32+8x1, 32+8x2, 32+8x3, ...
        'min': 32,
        'max': 256,
      }
    })

In [8]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 32,
                               'q': 8},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'values': [5, 10]},
                'fc_layer_size': {'values': [128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [9]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo-hw3")

Create sweep with ID: 5ohs6n3z
Sweep URL: https://wandb.ai/ymkim78/pytorch-sweeps-demo-hw3/sweeps/5ohs6n3z


In [10]:
device = torch.device("cpu")

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        loader = build_dataset(config.batch_size)
        network = build_network(config.fc_layer_size, config.dropout)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            wandb.log({"loss": avg_loss, "epoch": epoch})

In [11]:
def build_dataset(batch_size):

    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307,), (0.3081,))])
    # download MNIST training dataset
    dataset = datasets.MNIST('../data/mnist', train=True, download=True,
                             transform=transform)
    sub_dataset = torch.utils.data.Subset(
        dataset, indices=range(0, len(dataset), 5))
    loader = torch.utils.data.DataLoader(sub_dataset, batch_size=batch_size)

    return loader


def build_network(fc_layer_size, dropout):
    network = nn.Sequential(  # fully-connected, single hidden layer
        nn.Flatten(),
        nn.Linear(784, fc_layer_size), nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(fc_layer_size, 10),
        nn.LogSoftmax(dim=1))

    return network.to(device)


def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # ➡ Forward pass
        loss = F.nll_loss(network(data), target)
        cumu_loss += loss.item()

        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()

        wandb.log({"batch loss": loss.item()})

    return cumu_loss / len(loader)

In [12]:
wandb.agent(sweep_id, train, count=10)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


wandb: Agent Starting Run: 2rphb879 with config:
wandb: 	batch_size: 160
wandb: 	dropout: 0.3
wandb: 	epochs: 5
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.027065263772784676
wandb: 	optimizer: adam


batch loss,█▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▃▅▆█
loss,█▁▁▁▁
batch loss,0.44923
epoch,4
loss,0.51309


wandb: Agent Starting Run: bddd0rcr with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.5
wandb: 	epochs: 5
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.011959162466821216
wandb: 	optimizer: sgd


...

batch loss,█▅▃▄▃▃▃▂▃▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▁▁▂▂▁▂▁▁▁▂▁▁▁▁▁
epoch,▁▃▅▆█
loss,█▃▂▁▁
batch loss,0.25081
epoch,4
loss,0.11955


wandb: Agent Starting Run: twq8irqk with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.3
wandb: 	epochs: 5
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.00290806785852582
wandb: 	optimizer: adam


batch loss,█▄▃▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▁▁▂▂▂▂▂▂▁▂▂▁▂▂▂▁▂▁▂▁▁▁
epoch,▁▃▅▆█
loss,█▃▂▂▁
batch loss,0.14771
epoch,4
loss,0.09464


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fdc536ac0b0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fdc5354c5f0, execution_count=12 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fdc5354c170, raw_cell="wandb.agent(sweep_id, train, count=10)" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/home/student/db_hw_2/wandb_example.ipynb#X14sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

In [12]:
def train_continual(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        loaders = build_dataset_split(config.batch_size)
        network = build_network(config.fc_layer_size, config.dropout)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for loader in loaders:
            for epoch in range(config.epochs):
                avg_loss = train_epoch(network, loader, optimizer)
                wandb.log({"loss": avg_loss, "epoch": epoch})

In [13]:
def build_dataset_split(batch_size):

    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307,), (0.3081,))])
    # download MNIST training dataset
    dataset = datasets.MNIST('../data/mnist', train=True, download=True,
                             transform=transform)
    
    # split the dataset into 5 parts, each part with label with [0,1], [2,3], [4,5], [6,7], [8,9]
    sub_datasets = []
    for i in range(5):
        indices = [j for j in range(len(dataset)) if dataset[j][1] in [2*i, 2*i+1]]
        sub_datasets.append(torch.utils.data.Subset(dataset, indices))

    loaders = [torch.utils.data.DataLoader(sub_dataset, batch_size=batch_size) for sub_dataset in sub_datasets]
    return loaders

In [14]:
wandb.agent(sweep_id, train_continual, count=10)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


wandb: Agent Starting Run: u1vjjchc with config:
wandb: 	batch_size: 192
wandb: 	dropout: 0.3
wandb: 	epochs: 5
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.042007047589339014
wandb: 	optimizer: adam


batch loss,▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂█▂▂▂▂▂▂▂▆▂▂▂▂▂▂▂
epoch,▁▃▅▆█▁▃▅▆█▁▃▅▆█▁▃▅▆█▁▃▅▆█
loss,▂▁▁▁▁▇▂▂▂▂█▃▃▃▃▇▃▃▃▃█▃▃▃▃
batch loss,0.70245
epoch,4
loss,0.70166


wandb: Agent Starting Run: jmez332e with config:
wandb: 	batch_size: 88
wandb: 	dropout: 0.3
wandb: 	epochs: 5
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.01699405359463594
wandb: 	optimizer: adam


...

batch loss,▁▄▁▁▁▁▁▁▁▁█▂▁▃▂▂▁▃▁▁▁▁▂▁▃▂▁▃▁▁▁▁▆█▆▃▃▃▃▃
epoch,▁▃▅▆█▁▃▅▆█▁▃▅▆█▁▃▅▆█▁▃▅▆█
loss,▁▁▁▁▁▃▂▂▂▂▅▁▁▁▁▅▁▁▁▁█▄▃▂▂
batch loss,0.19398
epoch,4
loss,0.1473


wandb: Agent Starting Run: xrhob36k with config:
wandb: 	batch_size: 136
wandb: 	dropout: 0.5
wandb: 	epochs: 5
wandb: 	fc_layer_size: 128
wandb: 	learning_rate: 0.06020837233967597
wandb: 	optimizer: adam


batch loss,▁▁▂▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▆▂▂▂▂▂▂▂█▂▂▂▂▂▂▂
epoch,▁▃▅▆█▁▃▅▆█▁▃▅▆█▁▃▅▆█▁▃▅▆█
loss,▃▁▁▁▁▇▃▃▃▃█▃▃▃▃█▃▃▃▃█▃▃▃▃
batch loss,0.69673
epoch,4
loss,0.69542


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f3811ab75f0>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f381191fbf0, execution_count=14 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f381191d580, raw_cell="wandb.agent(sweep_id, train_continual, count=10)" store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell://wsl%2Bubuntu/home/student/db_hw_2/wandb_example.ipynb#X20sdnNjb2RlLXJlbW90ZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe